In [11]:
import pandas as pd
import numpy as np
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
data = pd.read_csv('data_caps2.csv')

In [3]:
data.head()
data.shape

,Unnamed: 0,pdbId,chain,resi,resn,phi,psi,is_cap,ALA,CYS,...,RESIDUE_CLASS2_IS_UNKNOWN_SHELL4,RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL5,RESIDUE_CLASS1_IS_CHARGED_SHELL5,RESIDUE_CLASS1_IS_POLAR_SHELL5,RESIDUE_CLASS1_IS_UNKNOWN_SHELL5,RESIDUE_CLASS2_IS_NONPOLAR_SHELL5,RESIDUE_CLASS2_IS_POLAR_SHELL5,RESIDUE_CLASS2_IS_BASIC_SHELL5,RESIDUE_CLASS2_IS_ACIDIC_SHELL5,RESIDUE_CLASS2_IS_UNKNOWN_SHELL5
0,0,2ygn,A,2,GLY,-66.66097,160.703190,0,0,0,...,-0.044339,-1.425341,-1.655148,-0.358659,0.819626,-1.506322,0.306017,-1.759420,-1.025543,-0.050447
1,1,2ygn,A,3,SER,-123.85361,-7.871733,0,0,0,...,-0.044339,-1.154537,-0.782982,0.477841,0.365701,-1.244529,1.033925,-1.233732,0.160914,-0.050447
2,2,2ygn,A,4,LEU,-74.89690,137.483930,0,0,0,...,-0.044339,0.876494,-0.782982,0.310541,0.819626,1.635192,0.487994,-1.759420,0.160914,-0.050447
3,3,2ygn,A,5,TYR,-134.41983,140.864290,0,0,0,...,-0.044339,-0.477527,-1.001024,1.481640,0.365701,0.457124,0.306017,-1.759420,-0.135700,-0.050447
4,4,2ygn,A,6,LEU,-139.27502,127.621544,0,0,0,...,-0.044339,0.199483,-1.873189,1.983539,-0.542149,1.242503,0.851948,-1.759420,-1.322158,-0.050447


(1561968, 83)

In [4]:
# number of unique pdbs
len(data.pdbId.unique())
# maximum seq length
data.groupby(['pdbId', 'chain']).pdbId.value_counts().max()
data.groupby(['pdbId', 'chain']).pdbId.value_counts().min()
counts = list(set(data.groupby(['pdbId', 'chain']).pdbId.value_counts()))
counts[:10]

6264

1269

2

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [5]:
# functions for converting phi psi angles to features
import numpy as np
def angle_to_cos(angle):
    if(angle == 0 or np.isnan(angle)):
        return 0
    else:
        return np.cos(np.pi * angle/180)

def angle_to_sin(angle):
    if(angle == 0 or np.isnan(angle)):
        return 0
    else:
        return np.sin(np.pi * angle/180)

In [6]:
# convert phi psi angles to sin cos
data['sin_phi'] = data.phi.apply(lambda x: angle_to_sin(x))
data['cos_phi'] = data.phi.apply(lambda x: angle_to_cos(x))
data['sin_psi'] = data.psi.apply(lambda x: angle_to_sin(x))
data['cos_psi'] = data.psi.apply(lambda x: angle_to_cos(x))
data.head()

,Unnamed: 0,pdbId,chain,resi,resn,phi,psi,is_cap,ALA,CYS,...,RESIDUE_CLASS1_IS_UNKNOWN_SHELL5,RESIDUE_CLASS2_IS_NONPOLAR_SHELL5,RESIDUE_CLASS2_IS_POLAR_SHELL5,RESIDUE_CLASS2_IS_BASIC_SHELL5,RESIDUE_CLASS2_IS_ACIDIC_SHELL5,RESIDUE_CLASS2_IS_UNKNOWN_SHELL5,sin_phi,cos_phi,sin_psi,cos_psi
0,0,2ygn,A,2,GLY,-66.66097,160.703190,0,0,0,...,0.819626,-1.506322,0.306017,-1.759420,-1.025543,-0.050447,-0.918177,0.396171,0.330462,-0.943819
1,1,2ygn,A,3,SER,-123.85361,-7.871733,0,0,0,...,0.365701,-1.244529,1.033925,-1.233732,0.160914,-0.050447,-0.830464,-0.557073,-0.136956,0.990577
2,2,2ygn,A,4,LEU,-74.89690,137.483930,0,0,0,...,0.819626,1.635192,0.487994,-1.759420,0.160914,-0.050447,-0.965459,0.260557,0.675797,-0.737088
3,3,2ygn,A,5,TYR,-134.41983,140.864290,0,0,0,...,0.365701,0.457124,0.306017,-1.759420,-0.135700,-0.050447,-0.714230,-0.699911,0.631159,-0.775653
4,4,2ygn,A,6,LEU,-139.27502,127.621544,0,0,0,...,-0.542149,1.242503,0.851948,-1.759420,-1.322158,-0.050447,-0.652429,-0.757850,0.792060,-0.610443


In [7]:
# drop the extra index column
data = data.drop(['Unnamed: 0'], axis=1)
data.head()
data.shape

,pdbId,chain,resi,resn,phi,psi,is_cap,ALA,CYS,ASP,...,RESIDUE_CLASS1_IS_UNKNOWN_SHELL5,RESIDUE_CLASS2_IS_NONPOLAR_SHELL5,RESIDUE_CLASS2_IS_POLAR_SHELL5,RESIDUE_CLASS2_IS_BASIC_SHELL5,RESIDUE_CLASS2_IS_ACIDIC_SHELL5,RESIDUE_CLASS2_IS_UNKNOWN_SHELL5,sin_phi,cos_phi,sin_psi,cos_psi
0,2ygn,A,2,GLY,-66.66097,160.703190,0,0,0,0,...,0.819626,-1.506322,0.306017,-1.759420,-1.025543,-0.050447,-0.918177,0.396171,0.330462,-0.943819
1,2ygn,A,3,SER,-123.85361,-7.871733,0,0,0,0,...,0.365701,-1.244529,1.033925,-1.233732,0.160914,-0.050447,-0.830464,-0.557073,-0.136956,0.990577
2,2ygn,A,4,LEU,-74.89690,137.483930,0,0,0,0,...,0.819626,1.635192,0.487994,-1.759420,0.160914,-0.050447,-0.965459,0.260557,0.675797,-0.737088
3,2ygn,A,5,TYR,-134.41983,140.864290,0,0,0,0,...,0.365701,0.457124,0.306017,-1.759420,-0.135700,-0.050447,-0.714230,-0.699911,0.631159,-0.775653
4,2ygn,A,6,LEU,-139.27502,127.621544,0,0,0,0,...,-0.542149,1.242503,0.851948,-1.759420,-1.322158,-0.050447,-0.652429,-0.757850,0.792060,-0.610443


(1561968, 86)

In [8]:
# drop phi psi columns as we already extracted features from these
data = data.drop(['phi', 'psi', 'resi', 'resn', 'residue'], axis=1)
data.head()

,pdbId,chain,is_cap,ALA,CYS,ASP,GLU,PHE,GLY,HIS,...,RESIDUE_CLASS1_IS_UNKNOWN_SHELL5,RESIDUE_CLASS2_IS_NONPOLAR_SHELL5,RESIDUE_CLASS2_IS_POLAR_SHELL5,RESIDUE_CLASS2_IS_BASIC_SHELL5,RESIDUE_CLASS2_IS_ACIDIC_SHELL5,RESIDUE_CLASS2_IS_UNKNOWN_SHELL5,sin_phi,cos_phi,sin_psi,cos_psi
0,2ygn,A,0,0,0,0,0,0,1,0,...,0.819626,-1.506322,0.306017,-1.759420,-1.025543,-0.050447,-0.918177,0.396171,0.330462,-0.943819
1,2ygn,A,0,0,0,0,0,0,0,0,...,0.365701,-1.244529,1.033925,-1.233732,0.160914,-0.050447,-0.830464,-0.557073,-0.136956,0.990577
2,2ygn,A,0,0,0,0,0,0,0,0,...,0.819626,1.635192,0.487994,-1.759420,0.160914,-0.050447,-0.965459,0.260557,0.675797,-0.737088
3,2ygn,A,0,0,0,0,0,0,0,0,...,0.365701,0.457124,0.306017,-1.759420,-0.135700,-0.050447,-0.714230,-0.699911,0.631159,-0.775653
4,2ygn,A,0,0,0,0,0,0,0,0,...,-0.542149,1.242503,0.851948,-1.759420,-1.322158,-0.050447,-0.652429,-0.757850,0.792060,-0.610443


In [9]:
# remove pdbs less than seq len 10
data_filtered = data[data.groupby(['pdbId', 'chain']).pdbId.transform(len) >= 10]
len(data_filtered.pdbId.unique())
data_filtered.shape

6246

(1561845, 81)

In [16]:
def generate_data(df_group, window_size = 10):
    
    columns = list(range(window_size * len(df_group.columns)))
    df_features = pd.DataFrame(columns=columns)
    
    for i in range(df_group.shape[0]):

        # if there are enough rows after current row, append these as columns
        if (i + window_size) <= df_group.shape[0]:

            row_start = group_data.iloc[i].to_frame().T    
            row_start = row_start.reset_index(drop=True)
            row_end = group_data.iloc[i+1:i+window_size].unstack().to_frame().T
            row_end_labels_as_str = [[str(j) for j in i] for i in row_end.columns.labels]
            row_end.columns = row_end.columns.get_level_values(0).astype(str).values + '_' + row_end_labels_as_str[1]

            merged_row = pd.merge(row_start, row_end, left_index=True, right_index=True)
            merged_row = merged_row.reindex(sorted(merged_row.columns), axis=1)

            df_features.columns = merged_row.columns
            df_features = df_features.append(merged_row, ignore_index=True)

        # else if there are enough rows before current row, append previous rows as columns
        elif ((i - window_size) >= 0):

            row_start = group_data.iloc[i].to_frame().T    
            row_start = row_start.reset_index(drop=True)
            row_end = group_data.iloc[i-window_size: i-1].unstack().to_frame().T
            row_end_labels_as_str = [[str(j) for j in i] for i in row_end.columns.labels]
            row_end.columns = row_end.columns.get_level_values(0).astype(str).values + '_' + row_end_labels_as_str[1]
            
            merged_row = pd.merge(row_start, row_end, left_index=True, right_index=True)
            merged_row = merged_row.reindex(sorted(merged_row.columns), axis=1)

            df_features = df_features.append(merged_row, ignore_index=True)

        # else take rows before and rows after
        else:

            row_start = group_data.iloc[i].to_frame().T    
            row_start = row_start.reset_index(drop=True)
            
            row_prev = group_data.iloc[0: i].unstack().to_frame().T
            row_next = group_data.iloc[i + 1: window_size].unstack().to_frame().T
            
            temp_list = row_next.columns.labels[1]
            temp_list += (max(row_prev.columns.labels[1]) + 1)
            
            row_prev_labels_as_str = [[str(j) for j in i] for i in row_prev.columns.labels]
            row_prev.columns = row_prev.columns.get_level_values(0).astype(str).values + '_' + row_prev_labels_as_str[1]

            row_next_labels_as_str = [[str(j) for j in i] for i in row_next.columns.labels]
            row_next.columns = row_next.columns.get_level_values(0).astype(str).values + '_' + row_next_labels_as_str[1]

            merged_row_ = pd.merge(row_start, row_prev, left_index=True, right_index=True)
            merged_row = pd.merge(merged_row_, row_next, left_index=True, right_index=True)
            merged_row = merged_row.reindex(sorted(merged_row.columns), axis=1)
            
            df_features = df_features.append(merged_row, ignore_index=True)
        
    return df_features

In [17]:
# debug
x = data_filtered[data_filtered.pdbId == '1jy2']
x = x.reindex(sorted(x.columns), axis=1)
x.shape
x.head()

df = x.groupby(['pdbId', 'chain'])

dict_df_features = {}

count = 0
for (pdbId, chain), group_data in df:
    start = time.time()
    dict_df_features[pdbId+chain] = generate_data(group_data)
    end = time.time()
    print(pdbId, ' ', chain, ' ', count, ' ', end - start)
    count += 1
    
    if count % 1000 == 0:
        print(pdbId, ' ', count, ' ', end - start)
        df_saved = pd.concat(dict_df_features.values(), ignore_index=True)
        df_saved.to_pickle("./" + str(count) + "_saved_master1.pkl")

# concat all dataframes in dict
df_feature_master = pd.concat(dict_df_features.values(), ignore_index=True)

df_feature_master.head()
df_feature_master.shape

(47, 81)

,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,...,TRP,TYR,VAL,chain,cos_phi,cos_psi,is_cap,pdbId,sin_phi,sin_psi
626944,0,0,0,0,0,0,0,0,0,0,...,1,0,0,N,0.293859,-0.926504,0,1jy2,-0.955849,0.376284
626945,0,0,0,0,0,0,0,0,0,0,...,0,0,0,N,0.317867,-0.910043,0,1jy2,-0.948135,0.414514
626946,0,0,0,0,0,0,0,0,0,0,...,0,0,0,N,0.349155,-0.825338,0,1jy2,-0.937065,0.564639
626947,0,0,0,0,1,0,0,0,0,0,...,0,0,0,N,0.305807,-0.927349,0,1jy2,-0.952094,0.374197
626948,0,0,0,0,0,0,0,0,0,0,...,0,0,0,N,-0.379241,-0.977526,0,1jy2,-0.925298,0.210816


1jy2   N   0   0.24096107482910156
1jy2   O   1   0.10305118560791016


,ALA,ALA_0,ALA_1,ALA_2,ALA_3,ALA_4,ALA_5,ALA_6,ALA_7,ALA_8,...,sin_psi,sin_psi_0,sin_psi_1,sin_psi_2,sin_psi_3,sin_psi_4,sin_psi_5,sin_psi_6,sin_psi_7,sin_psi_8
0,0,0,0,0,0,0,0,0,0,0,...,0.376284,0.414514,0.564639,0.374197,0.210816,-0.546785,-0.386577,-0.497633,0.675765,-0.283936
1,0,0,0,0,0,0,0,0,0,0,...,0.414514,0.564639,0.374197,0.210816,-0.546785,-0.386577,-0.497633,0.675765,-0.283936,-0.528546
2,0,0,0,0,0,0,0,0,0,0,...,0.564639,0.374197,0.210816,-0.546785,-0.386577,-0.497633,0.675765,-0.283936,-0.528546,0.742377
3,0,0,0,0,0,0,0,0,0,0,...,0.374197,0.210816,-0.546785,-0.386577,-0.497633,0.675765,-0.283936,-0.528546,0.742377,0.290666
4,0,0,0,0,0,0,0,0,0,0,...,0.210816,-0.546785,-0.386577,-0.497633,0.675765,-0.283936,-0.528546,0.742377,0.290666,0.301003


(47, 810)

In [ ]:
# generate feature data
df = data_filtered.groupby(['pdbId', 'chain'])

dict_df_features = {}

count = 0
for (pdbId, chain), group_data in df:
    start = time.time()
    dict_df_features[pdbId+chain] = generate_data(group_data)
    end = time.time()
    print(pdbId, ' ', chain, ' ', count, ' ', end - start)
    count += 1
    
    if count % 1000 == 0:
        print(pdbId, ' ', count, ' ', end - start)
        df_saved = pd.concat(dict_df_features.values(), ignore_index=True)
        df_saved.to_pickle("./" + str(count) + "_saved_master1.pkl")

# concat all dataframes in dict
df_feature_master = pd.concat(dict_df_features.values(), ignore_index=True)

1a1x   A   0   0.6687369346618652
1a62   A   1   0.7817168235778809
1a73   A   2   1.0392534732818604
1a8l   A   3   1.479212760925293
1a92   A   4   0.3211967945098877
1a9x   A   5   9.634728908538818
1ae9   A   6   1.0620157718658447
1ah7   A   7   1.615231990814209
1al3   A   8   0.9673826694488525
1aoc   A   9   1.131903886795044
1aol   A   10   1.4989545345306396
1arb   A   11   1.755192518234253
1atg   A   12   1.5205106735229492
1atz   A   13   1.179910659790039
1ayl   A   14   3.8532052040100098
1ayo   A   15   0.8297371864318848
1azo   A   16   1.2530450820922852
1b12   A   17   1.546464204788208
1b25   A   18   5.031758069992065
1b5e   A   19   1.5751621723175049
1b6a   A   20   1.519287109375
1baz   A   21   0.29497694969177246
1bea   A   22   0.7180662155151367
1bf2   A   23   6.155235052108765
1bgc   A   24   0.9257805347442627
1bgf   A   25   0.789088249206543
1bkr   A   26   0.6880741119384766
1bm8   A   27   0.6326141357421875
1brt   A   28   1.8541157245635986
1btk   A

MemoryError: 

In [ ]:
df_feature_master.to_pickle("./complete_master_ws10.pkl")